In [136]:
import pickle
import json
import urllib
import requests

In [2]:
BASE_PATH = 'E:\Datasets\TWITTER_SOCG\%s'
file_name = 'filtered_loc148.json'
file = BASE_PATH%(file_name)

In [113]:
def get_location_data(location_name):
    '''
    This function gets the location details for a given text location
    https://nominatim.openstreetmap.org/search.php?q=california&format=json&limit=1
    '''
    base_url = 'https://nominatim.openstreetmap.org/search.php?%s'
    search_term = {'format':'json','limit':1}
    search_term['q'] = location_name
    url_args = urllib.parse.urlencode(search_term)
    url = base_url%(url_args)
    response = requests.get(url)
    res = [{}]
    if response.status_code != 200:
        #Some error handling
        print("Unable to fetch data from API, Reponse = ",response.status_code)
    else:
        try:
            res = json.loads(response.content)
        except Exception as e:
            print("Error for content,",e)
    return res[0]

In [76]:
def fix_JSON(json_message=None):
    '''This function fixes errors in the JSON if any and return a json dictionary'''
    result = None
    try:        
        result = json.loads(json_message)
    except Exception as e:      
        # Find the offending character index:
        e_message = str(e)
        idx_to_replace = int(e_message.split(' ')[-1].replace(')',''))   
        
        #print(e_message,idx_to_replace)
        # Remove the offending character:
        json_message = list(json_message)
        json_message[idx_to_replace] = ' '
        new_message = ''.join(json_message)     
        return fix_JSON(json_message=new_message)
    return result

In [140]:
def data_filter(arg,params):
    res = {}
    for p in params:
        res[p] = arg[p]
    return res

In [70]:
data = {}

In [77]:
criteria = ['id','location','description','followers_count','friends_count'] #user filtering criteria

In [79]:
## Filtering user data
with open(file) as f:
    for line in f:
        tmp_all = fix_JSON(line)
        user = tmp_all['user'] 
        data[user['id']] = data_filter(user,criteria)

In [139]:
print("SIZE of data = ",len(data))

SIZE of data =  18196


In [117]:
locations = {}

In [ ]:
## Getting all location name details
for k,v in data.items():
    if k not in locations:
        location_name = v.get('location',None)
        if location_name is not None:
            locations[location_name] = get_location_data(location_name)

In [138]:
## Dumping location data
with open('location_by_user.pkl', 'wb') as handle:
    pickle.dump(locations, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [134]:
## Generating data for plotting
coord = []
for k,v in locations.items():
    if len(v) != 0:
        coord.append((float(v[0]['lat']),float(v[0]['lon'])))

In [142]:
## Dumping coordinate data
with open('coord_user_1.pkl', 'wb') as handle:
    pickle.dump(coord, handle, protocol=pickle.HIGHEST_PROTOCOL)